In [1]:
pip install --upgrade kiwipiepy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 5.0 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.7/34.7 MB 4.1 MB/s eta 0:00:00:00:0100:01
  Preparing metadata (setup.py) ... done
  Attempting uninstall: kiwipiepy-model
    Found existing installation: kiwipiepy-model 0.19.0
    Uninstalling kiwipiepy-model-0.19.0:
      Successfully uninstalled kiwipiepy-model-0.19.0
  DEPRECATION: kiwipiepy-model is being installed using the legacy 'setup.py install' method, because it does not have a 'pyproject.toml' and the 'wheel' package is not installed. pip 23.1 will enforce this behaviour change. A possible replacement is to enable the '--use-pep517' option. Discussion can be found at https://github.com/pypa/pip/issues/8559
  Running setup.py install for kiwipiepy-model ... done
  Attempting uninstall: kiwipiepy
    Found existing installation: kiwipiepy 0.19.0
    Uninstalling kiwipiepy-0.19.0:
      Successfully uninstalled kiwipiepy-0.19.0

[notice] 

In [1]:
# pip install kiwipiepy==0.19.0

  Using cached kiwipiepy-0.18.1-cp311-cp311-win_amd64.whl.metadata (1.1 kB)
  Using cached kiwipiepy_model-0.18.0-py3-none-any.whl
Using cached kiwipiepy-0.18.1-cp311-cp311-win_amd64.whl (1.3 MB)
  Attempting uninstall: kiwipiepy-model
    Found existing installation: kiwipiepy_model 0.19.0
    Uninstalling kiwipiepy_model-0.19.0:
      Successfully uninstalled kiwipiepy_model-0.19.0
  Attempting uninstall: kiwipiepy
    Found existing installation: kiwipiepy 0.19.0
    Uninstalling kiwipiepy-0.19.0:
      Successfully uninstalled kiwipiepy-0.19.0
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import openai # 0.28.0
import re
from kiwipiepy import Kiwi

# Kiwi 초기화
kiwi = Kiwi(num_workers=0, model_path=None, load_default_dict=True, integrate_allomorph=True, model_type='sbg', typos=None, typo_cost_threshold=2.5)

"""
num_workers: 2 이상이면 단어 추출 및 형태소 분석에 멀티 코어를 활용하여 조금 더 빠른 속도로 분석을 진행할 수 있습니다.
1인 경우 단일 코어만 활용합니다. num_workers가 0이면 현재 환경에서 사용가능한 모든 코어를 활용합니다.
생략 시 기본값은 0입니다.
model_path: 형태소 분석 모델이 있는 경로를 지정합니다. 생략시 kiwipiepy_model 패키지로부터 모델 경로를 불러옵니다.
load_default_dict: 추가 사전을 로드합니다. 추가 사전은 위키백과의 표제어 타이틀로 구성되어 있습니다. 이 경우 로딩 및 분석 시간이 약간 증가하지만 다양한 고유명사를 좀 더 잘 잡아낼 수 있습니다. 분석 결과에 원치 않는 고유명사가 잡히는 것을 방지하려면 이를 False로 설정하십시오.
integrate_allomorph: 어미 중, '아/어', '았/었'과 같이 동일하지만 음운 환경에 따라 형태가 달라지는 이형태들을 자동으로 통합합니다.
model_type: 형태소 분석에 사용할 언어 모델을 지정합니다. 'knlm', "sbg" 중 하나를 선택할 수 있습니다. "sbg" 는 상대적으로 느리지만 먼 거리에 있는 형태소 간의 관계를 포착할 수 있습니다.
typos: 형태소 분석 시 간단한 오타를 교정합니다. None으로 설정 시 교정을 수행하지 않습니다.
typo_cost_threshold: 오타 교정을 허용할 최대 오타 비용을 설정합니다.
"""

# API 키 설정
openai.api_key = "sk-proj-nlhN73CnCzO3ShLYyCPuT3BlbkFJdzOuNYCbeHCwAhrhhh7p"

        
######## 데이터 => src/v1/resoures/ 디렉토리 내부로 이동시켜서 불러오게 바꾸기
pos_descriptions  = {"NNG"	:"일반 명사",
"NNP"	:"고유 명사",
"NNB"	:"의존 명사",
"NR"	:"수사",
"NP"	:"대명사",
"VV"	:"동사",
"VV-R"	:"동사(규칙)",
"VV-I"	:"동사(불규칙)",
"VA"	:"형용사",
"VA-R"	:"형용사(규칙)",
"VA-I"	:"형용사(불규칙)",
"VX"	:"보조 용언",
"VX-R"	:"보조 용언(규칙)",
"VX-I"	:"보조 용언(불규칙)",
"VCP"	:"긍정 지시사(이다)",
"VCN"	:"부정 지시사(아니다)",
"MM"	:"관형사",
"MAG"	:"일반 부사",
"MAJ"	:"접속 부사",
"IC"	:"감탄사",
"JKS"	:"주격 조사",
"JKC"	:"보격 조사",
"JKG"	:"관형격 조사",
"JKO"	:"목적격 조사",
"JKB"	:"부사격 조사",
"JKV"	:"호격 조사",
"JKQ"	:"인용격 조사",
"JX"	:"보조사",
"JC"	:"접속 조사",
"EP"	:"선어말 어미",
"EF"	:"종결 어미",
"EC"	:"연결 어미",
"ETN"	:"명사형 전성 어미",
"ETM"	:"관형형 전성 어미",
"XPN"	:"체언 접두사",
"XSN"	:"명사 파생 접미사",
"XSV"	:"동사 파생 접미사",
"XSA"	:"형용사 파생 접미사",
"XSA-R"	:"형용사 파생 접미사(규칙)",
"XSA-I"	:"형용사 파생 접미사(불규칙)",
"XSM"	:"부사 파생 접미사",
"XR"	:"어근",
"SF"	:"종결 부호(. ! ?)",
"SP"	:"구분 부호(, / : ;)",
"SS"	:"""인용 부호 및 괄호(" " ( ) [ ] < > { } ― ‘ ’ “ ” ≪ ≫ 등)""",
"SSO"	:"SS 중 여는 부호",
"SSC"	:"SS 중 닫는 부호",
"SE"	:"줄임표(…)",
"SO"	:"붙임표(- ~)",
"SW"	:"기타 특수 문자",
"SL"	:"알파벳(A-Z a-z)",
"SH"	:"한자",
"SN"	:"숫자(0-9)",
"SB"	:"순서 있는 글머리(가. 나. 1. 2. 가) 나) 등)*",
"UN"	:"분석 불능",
"W_URL"	:"URL 주소",
"W_EMAIL"	:"이메일 주소",
"W_HASHTAG"	:"해시태그(#abcd)",
"W_MENTION"	:"멘션(@abcd)",
"W_SERIAL"	:"일련번호(전화번호, 통장번호, IP주소 등)",
"Z_CODA"	:"덧붙은 받침",
"USER0_4"	:"사용자 정의 태그"}



# 문법 항목 리스트 생성
"""
"/" 로 끊어서 판단.
품사번호 확인 후 형태가 같으면 해당 태그가 사용 됨.

"""


In [15]:
grammatical_items = [
    {
        "번호": 1,
        "형태": '라고 합니다',
        '품사': 'VCP+EC+VV+EF',
        '의미': '((‘이다’, ‘아니다’의 어간이나 어미 ‘-으시-’ 뒤에 붙어)) ‘이라’, ‘아니라’의 어미 ‘-라’에 격 조사 ‘고’가 결합한 말. 간접적으로 인용됨을 나타낸다.',
    },
    {
        '번호': 2,
        '형태': '있어요',
        '품사': 'VA+EF',
        '의미': '문장 속에서 어떤 대상이 화제임을 나타내는 보조사',
    },
    {
        '번호': 3,
        '형태': '안',
        '품사': 'MAG',
        '의미': '부정이나 반대의 뜻을 나타내는 말',
    },
    {
        '번호': 4,
        '형태': '에',
        '품사': 'JKB',
        '의미': '앞말이 처소의 부사어임을 나타내는 격 조사 또는 진행 방향의 부사어임을 나타내는 격 조사',
    },
    {
        '번호': 5,
        '형태': '-았/었/였어요',
        '품사': 'VV+EP+EF',
        '의미': "어떤 행동이 과거에 이루어졌음을 나타내는 말."
    },

    {
        '번호': 98,
        '형태': '에',
        '품사': 'JKB',
        '의미': '앞말이 시간의 부사어임을 나타내는 격 조사',
    },
    {
        '번호': 99,
        '형태': '에',
        '품사': 'JKB',
        '의미': '앞의 체언을 부사어가 되게 하는 조사',
    },
]

grammatical_items

[{'번호': 1,
  '형태': '라고 합니다',
  '품사': 'VCP+EC+VV+EF',
  '의미': '((‘이다’, ‘아니다’의 어간이나 어미 ‘-으시-’ 뒤에 붙어)) ‘이라’, ‘아니라’의 어미 ‘-라’에 격 조사 ‘고’가 결합한 말. 간접적으로 인용됨을 나타낸다.'},
 {'번호': 2, '형태': '있어요', '품사': 'VA+EF', '의미': '문장 속에서 어떤 대상이 화제임을 나타내는 보조사'},
 {'번호': 3, '형태': '안', '품사': 'MAG', '의미': '부정이나 반대의 뜻을 나타내는 말'},
 {'번호': 4,
  '형태': '에',
  '품사': 'JKB',
  '의미': '앞말이 처소의 부사어임을 나타내는 격 조사 또는 진행 방향의 부사어임을 나타내는 격 조사'},
 {'번호': 5,
  '형태': '-았/었/였어요',
  '품사': 'VV+EP+EF',
  '의미': '어떤 행동이 과거에 이루어졌음을 나타내는 말.'},
 {'번호': 98, '형태': '에', '품사': 'JKB', '의미': '앞말이 시간의 부사어임을 나타내는 격 조사'},
 {'번호': 99, '형태': '에', '품사': 'JKB', '의미': '앞의 체언을 부사어가 되게 하는 조사'}]

In [16]:


# 로직컬 적으로 판단을 위해
 
# 시간 명사
time_nouns = {
"시간", "때", "시각", "분", "초", "어제", "오늘", "내일", "모레", "그제", "글피", "지난주", "이번주", "다음주",
"월요일", "화요일", "수요일", "목요일", "금요일", "토요일", "일요일", "주말", "평일", "아침", "점심", "저녁", "밤", "새벽", "낮",
"방금", "그때", "이때", "저때", "지금", "그동안", "이후", "이전", "지난달", "이번달", "다음달", "작년", "올해", "내년",
"봄", "여름", "가을", "겨울", "어린 시절", "청소년기", "성인기", "노년기", "초", "중", "말", "첫날", "마지막 날",
"전날", "이튿날", "사흘", "나흘", "닷새", "엿새", "일주일", "보름", "한 달", "두 달", "세 달", "석 달", "반년", "일년", "두 해", "세 해",
"연초", "연말", "중순", "상반기", "하반기", "사계절", "1분기", "2분기", "3분기", "4분기", "10년", "세기", "밀레니엄", "대공황",
"유년기", "사춘기", "청년기", "중년기", "노년기", "이전", "다음", "이후", "현재", "과거", "미래", "선사시대", "고대", "중세", "근세", "현대",
"새벽 1시", "새벽 2시", "오전 3시", "오전 4시", "오후 5시", "오후 6시", "밤 7시", "밤 8시", "자정", "정오", 
"초반", "중반", "후반", "초기", "중기", "말기", "하루", "주중", "주간", "연중", "평생", "영원", "순간", "찰나", "잠시", "얼마간", 
"하루하루", "해마다", "계속", "영겁", "어느 날", "어느 순간", "동이 틀 무렵", "해질 녘", "밤중", "한낮", "정오", "이른 아침", "늦은 저녁",
"초저녁", "심야", "한밤중", "말년", "초창기", "말미", "막바지", "오랜만", "잠깐", "순식간", "한순간", "짧은 시간", "긴 시간",
"첫 번째", "두 번째", "세 번째", "네 번째", "다섯 번째", "처음", "마지막", "종반", "결말", "피날레",
"20세기", "21세기", "100년", "200년", "천년", "십년", "반세기", "한 세대", "두 세대", "역사 속에서", "오래전", 
"아득한 옛날","시간", "때", "시각", "분", "초", "어제", "오늘", "내일", "모레", "지난주", "이번주", "다음주",
"월요일", "화요일", "수요일", "목요일", "금요일", "토요일", "일요일", "주말", "아침", "점심", "저녁", "밤", "새벽", "방금", "그때", "지난달",
"이번달", "다음달", "작년", "올해", "내년", "저번" , "이번", "옛날", "새벽","아침","오전","자정","주","오전","백년","십년","일년",
"오후","오후 1시","오후 2시","오후 3시","오후 4시","오후 5시","오후 6시","오후 7시","오후 8시","오후 9시","오후 10시","오후 11시","오후 12시",
"오전","오전 1시","오전 2시","오전 3시","오전 4시","오전 5시","오전 6시","오전 7시","오전 8시","오전 9시","오전 10시","오전 11시","오전 12시",
"밤","밤 1시","밤 2시","밤 3시","밤 4시","밤 5시","밤 6시","밤 7시","밤 8시","밤 9시","밤 10시","밤 11시","밤 12시",
"낮","낮 1시","낮 2시","낮 3시","낮 4시","낮 5시","낮 6시","낮 7시","낮 8시","낮 9시","낮 10시","낮 11시","낮 12시",
"봄에","여름에","가을에","겨울에","초봄에","늦가을에","초여름에","한여름에","연말에","연초에","연중에","계절에","새벽에","아침에","점심에","저녁에","밤에","자정에",
"정오에","해질녘에","동틀 무렵에","오후에","이른 아침에","한낮에","늦은 밤에","내일","모레","다음 주에","다음 달에","내년에","미래에","앞으로","곧","장차","머지않아",
"훗날에","언젠가","장래에","이후에","곧바로","차후에","나중에","몇 년 후에","먼 훗날에","지금","현재","이 시간에","오늘","요즘에","요새","금방","방금","당장","오늘 아침에",
"이번 주에","올해","이번 달에","이번 학기에","이번에","이번 시기에","최근에","옛날","옛적에","예전에","과거에","지난달에","작년에","어제","그제","지난주에","한참 전에","몇 년 전에",
"어릴 때","유년 시절에","어린 시절에","사춘기 때","중학교 때","고등학교 때","대학교 때","어렸을 때","이전에","과거 어느 날","한때","초창기에","초기에","아득한 옛날에"
}

# 수량 명사
quantity_nouns = {
    '한', '두', '세', '네', '다섯', '여섯', '일곱', '여덟', '아홉', '열', '백', '천', '만',
    '개', '명', '번', '시간', '일', '주', '달', '년', '원', '킬로그램', '그램', '미터', '센티미터', '리터', '도'
}

place_nouns = {
    # 장소 명사
    "집", "학교", "회사", "공원", "도서관", "식당", "병원", "은행", "시장", "백화점",
    "마트", "극장", "역", "공항", "호텔", "카페", "레스토랑", "교실", "방", "거리",
    "도로", "바다", "산", "섬", "도시", "나라", "지하철", "버스정류장", "주차장",
    "화장실", "사무실", "교회", "성당", "절", "박물관", "미술관", "경찰서", "소방서",
    "체육관", "운동장", "수영장", "고속도로", "터미널", "아파트", "빌딩", "건물",
    "영화관", "놀이공원", "공연장", "전시장", "서점", "약국", "미용실", "음식점",
    "카센터", "주유소", "헬스장", "노래방", "카페테리아", "식품점", "편의점", "슈퍼마켓",
    "학교", "회사", "집", "교실", "강의실", "연구실", "실험실", "운전면허시험장",
    "운전학원", "운동장", "놀이방", "어린이집", "유치원", "초등학교", "중학교", "고등학교", "대학교","시골"
    "집","학교","회사","공원","병원","도서관","식당","극장","서점","카페","시장","백화점","마트","공장","경찰서","소방서","은행","우체국","박물관","미술관","체육관","공항","호텔","산","바다","강","호수","숲","해변","사막","동굴","계곡","들판","섬","폭포","초원","역","버스 정류장","지하철역","터미널","항구","주차장","정류장","고속도로","톨게이트",
    "아파트","빌딩","교회","성당","사찰","궁전","성","운동장","야구장","축구장","수영장","놀이공원","경기장",
    "한국","미국","일본","중국","영국","서울","부산","뉴욕","런던","파리","베이징","도쿄","베를린","로마","시드니",
    "화장실","교실","강의실","운전면허시험장","회의실","사무실","운전석","부엌","거실","방",

    # 진행 방향을 나타내는 명사
    "앞", "뒤", "옆", "위", "아래", "안", "밖", "사이", "중간", "주변", "근처",
    "오른쪽", "왼쪽", "동쪽", "서쪽", "남쪽", "북쪽", "내부", "외부", "방향", "쪽",
    "방면", "코너", "목적지", "출발지", "도착지", "지점", "위치", "장소", "경로",
    "길", "골목", "입구", "출구", "교차로", "사거리", "삼거리", "기차역", "버스터미널",
    "항구", "맞은편", "정면", "후면", "상단", "하단", "중앙", "가운데", "주위", "둘레",
    "바깥", "안쪽", "외곽", "부근", "접근", "도로", "차선", "차로", "이정표", "교차점",
    "코앞", "뒤편", "옆쪽", "바로", "곧장", "직진", "우회전", "좌회전", "유턴", "골목길",
    "샛길", "지름길", "고가도로", "지하도", "육교", "다리", "터널", "고개", "언덕","쪽"
}

In [17]:
kiwi.analyze("테스트 텍스트 입니다.")

[([Token(form='테스트', tag='NNG', start=0, len=3),
   Token(form='텍스트', tag='NNG', start=4, len=3),
   Token(form='이', tag='VCP', start=8, len=1),
   Token(form='ᆸ니다', tag='EF', start=8, len=3),
   Token(form='.', tag='SF', start=11, len=1)],
  -33.727684020996094)]

In [18]:
# 함수 내에서 동기적으로 호출
def chatgpt(text, matched_items, pna_token, now_token):
    response = openai.ChatCompletion.create(
        model="gpt-4", #gpt-4 ,gpt-4o-mini , gpt-4o, gpt-3.5-turbo
        messages=[{
            "role": "user",
            "content": f"""
            sentence: {text}
            pna_token : {pna_token}
            now_token : {now_token}
            matched_items: {matched_items}

            현재 나는 한국어 문장 태깅에서 판단하기 어려운 부분을 너에게 맡기려고 해.
            sentence는 전체 문장이고, pna_token의 가운데인 now_token의 태깅을 판단해야해.
            matched_items는 현재 태깅의 어려운 부분이야. 이 부분이 2가지 이상의 태깅이 되어있어.
            matched_items의 번호가 몇번이 정답일지 판단해서 숫자로 정답만 알려줘
            matched_items의 번호가 0번인 경우 nlp모델이 분류한 결과야 이게 맞을 수 있는데 나머지 번호가 틀렸다고 판단될 경우 0번으로 알려줘
            그렇지 않은 경우 반드시 matched_items의 번호 중에서의 숫자만 알려줘.
            반드시 설명하지말고 숫자만 알려줘.
            """
        }],
        stream=False,  # 스트리밍 비활성화
    )

    # 응답 처리
    full_text = str(response['choices'][0]['message']['content'])
    print("====GPT answer====", full_text)

    return full_text

In [19]:
grammatical_items_dict = {item['번호']: item for item in grammatical_items} # 딕셔너리로 빨리 찾기
# token_1 = grammatical_items_dict[n] # n번 태깅 찾기

In [20]:
change_pos_descriptions = [grammatical_items[k] for k in range(len(grammatical_items)) if (grammatical_items[k]['번호']==16)][0] # 16번 태깅 찾기
grammatical_items_dict[16] # n번 태깅 찾기


IndexError: list index out of range

In [ ]:
import tiktoken

# 사용하는 모델에 맞는 인코더 선택 (예: gpt-3.5-turbo)
encoding = tiktoken.get_encoding("cl100k_base")

# 텍스트를 토큰으로 변환
text = "여기에 계산하고 싶은 텍스트를 입력하세요."
tokens = encoding.encode(text)

# 토큰의 개수 출력
print(len(tokens))

17


In [7]:
# #############
# # 
# def pos_tag_print(results):
#     output = []
#     # 형태소 분석 결과와 문법 항목 매핑
#     for tokens, score in results:
#         for token in tokens:
#             morpheme = token.form  # 형태소 (form)
#             pos = token.tag        # 품사 태그 (tag)
#             pos_desc = pos_descriptions.get(pos, '알 수 없음')
#             matched_items = []
#             for item in grammatical_items:
#                 # 형태에서 '-'를 제거하고 '/'로 분리하여 각 형태소를 비교
#                 item_forms = item['형태'].replace('-', '').split('/')
#                 if morpheme in item_forms and pos == item['품사']:
#                     matched_items.append(item)
#             output.append({
#                 'morpheme': morpheme,
#                 'pos': pos,
#                 'pos_desc': pos_desc,
#                 'matched_items': matched_items
#             })
#     return output

def pos_tag_print(results):
    output = []
    # 형태소 분석 결과와 문법 항목 매핑
    for tokens, score in results:
        for token in tokens:
            morpheme = token.form  # 형태소 (form)
            pos = token.tag        # 품사 태그 (tag)
            start = token.start    # 형태소의 시작 인덱스
            length = token.len     # 형태소의 길이
            end = start + length   # 형태소의 끝 인덱스 계산
            pos_desc = pos_descriptions.get(pos, '알 수 없음')
            matched_items = []
            for item in grammatical_items:
                # 형태에서 '-'를 제거하고 '/'로 분리하여 각 형태소를 비교
                item_forms = item['형태'].replace('-', '').split('/')
                if morpheme in item_forms and pos == item['품사']:
                    matched_items.append(item)
            output.append({
                'morpheme': morpheme,
                'pos': pos,
                'pos_desc': pos_desc,
                'matched_items': matched_items,
                'start': start,
                'len': length,
                'end': end
            })
    return output

In [22]:
#문장을 끊어서 판단
def split_sentences(output): 
    sentences = []
    sentence = []
    for token in output: 
        sentence.append(token)
        if token['pos'] in ['SF']: # 문장 구분: 종결 어미(EF)와 종결 부호(SF)를 기준으로 문장을 분리 # @@ 바꾸어야 할 지도.
            sentences.append(sentence)
            sentence = []
    # 마지막 문장이 EF나 SF로 끝나지 않는 경우 처리
    if sentence:
        sentences.append(sentence)
    return sentences


def check_logic(sentence_tokens):
    
    output = all_text = sentence_tokens # GPT에 쓰임
    assigned_indices = set()
    
    ### JKB 처리 (10번 11번)
    for idx, token in enumerate(output):
        if token['morpheme'] == '에' and token['pos'] == 'JKB':
            assigned = False  # 번호가 할당되었는지 여부를 추적
            # 앞의 토큰을 가져옵니다.
            if idx > 0:
                # '에' 앞의 가장 가까운 명사(N)를 찾아 시간 표현인지 확인
                for back_idx in range(idx - 1, -1, -1):
                    back_token = output[back_idx]
                    # print(back_token)  # 디버깅용 출력
                    if back_token['pos'].startswith('N'):
                        print(back_token['morpheme'])  # 디버깅용 출력
                        
                        
                        if back_token['morpheme'] in time_nouns:
                            print("11번", back_token['morpheme'])  # 디버깅용 출력
                            # 시간 표현 단어를 찾았으므로 번호 11로 설정
                            item11 = next((item for item in token['matched_items'] if item['번호'] == 11), None)
                            if item11:
                                token['matched_items'] = [item11]
                                break  # 가장 가까운 N*를 찾았으므로 루프 종료

                        if back_token['morpheme'] in place_nouns:
                            print("10번", back_token['morpheme'])  # 디버깅용 출력
                            # 처소 및 진행 방향의 부사어 인지 판단하는 로직 추가
                            item10 = next((item for item in token['matched_items'] if item['번호'] == 10), None)
                            if item10:
                                token['matched_items'] = [item10]
                                break  # 가장 가까운 N*를 찾았으므로 루프 종료
                        else:
                            # 가장 가까운 명사가 시간이나 장소가 아니니 태깅 x
                            break
            else:
                # 앞의 토큰이 없는 경우 번호를 할당하지 않음
                pass
            

    ### '나/이나' (JC) 처리: 아이템 21번 태깅
    for idx, token in enumerate(output):
        if token['morpheme'] in ['나', '이나'] and token['pos'] == 'JC':
            # 앞뒤 토큰을 확인합니다.
            prev_token = output[idx - 1] if idx > 0 else None
            next_token = output[idx + 1] if idx + 1 < len(output) else None
            nnext_token = output[idx + 2] if idx + 2 < len(output) else None

            if prev_token and next_token:
                # 앞뒤 토큰의 품사가 명사(N*)인지 확인
                if prev_token['pos'].startswith('N') and next_token['pos'].startswith('N') and (nnext_token['pos'].endswith('V') == False):
                    # 번호 21번 항목을 가져옵니다.
                    token_1 = grammatical_items_dict[21] # n번 태깅 찾기
                    token['pos']= token_1["품사"]
                    token['pos_desc']= token_1["의미"]
                    token['matched_items'].append(token_1)
        
    ### '세요' (EF) 처리  17번 (강제로 찾게 만듦.)
    for token in output:
        if token['pos'] == 'EF' and token['morpheme'].endswith('세요'):

            token_1 = token_1 = grammatical_items_dict[17] # n번 태깅 찾기
            token['pos']= token_1["품사"]
            token['pos_desc']= token_1["의미"]
            token['matched_items'].append(token_1)
    
    # '수' + '밖' + '에' 패턴 병합 로직 추가
    idx = 0
    while idx < len(output) - 2:
        prev_token = output[idx]
        token = output[idx + 1]
        next_token = output[idx + 2]
        # '수'(NNB) + '밖'(NNG) + '에'(JKB) 패턴 확인
        if (prev_token['morpheme'] == '수' and prev_token['pos'] == 'NNB' and
            token['morpheme'] == '밖' and token['pos'] == 'NNG' and
            next_token['morpheme'] == '에' and next_token['pos'] == 'JKB'):
            # '밖' + '에'를 병합하여 '밖에'로 만듦
            merged_token = {
                'morpheme': token['morpheme'] + next_token['morpheme'],
                'pos': token_1["품사"],
                'pos_desc': token_1["의미"],
                'matched_items': [token_1],
                'start': token['start'],
                'len': token['len'] + next_token['len'],
                'end': token['start'] + token['len'] + next_token['len']
            }
            # 문법 항목 번호 24번 적용
            item24 = grammatical_items_dict[24]  # 번호 24번 항목
            if item24:
                merged_token['matched_items'].append(item24)
            # '수'는 그대로 두고 '밖'과 '에'를 병합된 토큰으로 대체
            output = output[:idx + 1] + [merged_token] + output[idx + 3:]
            # 인덱스 조정
            idx += 1
        else:
            idx += 1

            
    ### 16번 태깅 '보' + '고' 패턴 병합 로직 추가
    idx = 0
    while idx < len(output) - 1:
        token = output[idx]
        next_token = output[idx + 1]
        if token['morpheme'] == '보' and token['pos'] == 'VV' and next_token['morpheme'] == '고' and next_token['pos'] == 'EC':
            token_1 = grammatical_items_dict[16] # n번 태깅 찾기
            
            print(token_1)
            # '보' + '고'를 병합하여 '보고'로 만듦
            merged_token = {
                'morpheme': token['morpheme'] + next_token['morpheme'],
                'pos': token_1["품사"],
                'pos_desc': token_1["의미"],
                'matched_items': [token_1],
                'start': token['start'],
                'len': token['len'] + next_token['len'],
                'end': token['start'] + token['len'] + next_token['len']
            }
            # 기존 토큰을 삭제하고 병합된 토큰을 삽입
            output = output[:idx] + [merged_token] + output[idx + 2:]
            # 인덱스 조정
            if idx > 0:
                idx -= 1  # 병합 후 이전 위치로 이동하여 앞의 토큰과의 관계도 검사
            else:
                idx = 0
        else:
            idx += 1
            
    # N* N* N* 패턴에서 가운데 N이 '보고'인 경우 번호 16번 태깅 (강제로 찾게 만듦.)
    for idx in range(1, len(output) - 1):
        token = output[idx]
        if token['pos'] == 'NNG' and token['morpheme'] == '보고':
            prev_token = output[idx - 1]
            next_token = output[idx + 1]
            if prev_token['pos'].startswith('N') and next_token['pos'].startswith('N'):
                # 번호 16으로 설정
                token_1 = grammatical_items_dict[16] # n번 태깅 찾기
                token['pos']= token_1["품사"]
                token['pos_desc']= token_1["의미"]
                token['matched_items'].append(token_1)


    # gpt 이용 부문.
    # @@ 여기서 gpt가 답을 고르지 못한 경우 None 태그를 만들어서 이 경우엔 기본 태그가 나오게 하자.
    # 최종, 태그가 2개 이상 있는 경우 해당 태그를 llm모델에 맡겨서 뭐가 맞는지 판단하게 함.
    for idx, token in enumerate(output):
        if len(token["matched_items"]) >= 2:
            # 이전 토큰과 다음 토큰을 가져옵니다.
            pre_token = output[idx - 1]["morpheme"] if idx > 0 else ''
            now_token = token["morpheme"]
            after_token = output[idx + 1]["morpheme"] if idx + 1 < len(output) else ''

            # 문장 전체 텍스트를 재구성합니다.
            sentence_text = ''.join(tok['morpheme'] for tok in output)

            # try:
            #     # GPT 함수 호출
            #     token['matched_items'].insert(0,{'번호': 0,'형태': token['morpheme'],'품사': token['pos'],'의미': token['pos_desc']}) # 기존 형태의 품사(nlp가 분류한 더 큰 집합[분류]의 품사)를 넣고 비교
            #     print(token['matched_items'])
            #     gpt_answer = chatgpt(sentence_text, token["matched_items"], pre_token + now_token + after_token, now_token)
                
            #     correct_tag = int(re.findall(r'\d+', gpt_answer)[0]) # GPT 응답에서 숫자 추출
            #     print(correct_tag,"correct_tag")

            #     item = next((item for item in token['matched_items'] if item['번호'] == correct_tag), None)
            #     if item:
            #         token['matched_items'] = [item]

            #     print("llm사용됨", sentence_text)
            #     print("정답으로 만든 태그", correct_tag, token["matched_items"])

            # except Exception as e:
            #     correct_tag = 0
            #     print("GPT오류", e)
            #     pass    
            
            if correct_tag == 0: # 0번의 경우 그대로 태깅
                token['matched_items'] = []
            else:
                item = next((item for item in token['matched_items'] if item['번호'] == correct_tag), None)
                if item:
                    token['matched_items'] = [item]
            
        else:
            # 태그가 1개인 경우 패스 
            pass
        
    # 최종 결과 출력
    for token in output:
        morpheme = token['morpheme']
        pos = token['pos']
        pos_desc = token['pos_desc']
        start = token['start']
        length = token['len']
        matched_items = token['matched_items']

        if matched_items and (matched_items[0]['번호'] != 0):
            for item in matched_items:
                print(f"형태소 '{morpheme}' (위치: {start}, 길이: {length}, 품사: {pos} - {pos_desc})는 문법 항목 번호 {item['번호']}에 해당합니다: {item['의미']}")
        else:
            print(f"형태소 '{morpheme}' (위치: {start}, 길이: {length}, 품사: {pos} - {pos_desc})")
            
    return output  # 수정된 output 리스트 반환"""

def main(text):
    # pos_tag_print 함수 호출 # 형태소 분석 수행
    output = pos_tag_print(results = kiwi.analyze(text))
    # 문장 단위로 분리 @@ => 이붑분 kiwi 분석기가 있다했음
    
    sentences = split_sentences(output)
    # 각 문장별로 처리하고 결과를 합침

    final_output = []
    for sentence_tokens in sentences:
        processed_sentence = check_logic(sentence_tokens)
        final_output.extend(processed_sentence)
        
    return final_output


In [23]:
# 잘 틀리는 것 예시 잘 못하는

text = """
우리 할아버지는 연세에 비하면 젊어 보이세요.
햄버거에 비하면 샌드위치는 칼로리가 낮은 편이에요.
"""

final_output = main(text)
final_output

연세


KeyError: 17

In [24]:
# 2번 예시
text = """
책은 서재에 있어요.
거거엔 뭐가 있어요?
집 근처에 예쁜 카페가 있어요.
여기에서 버스를 탈 수 있는 정류장이 있어요?
"""
final_output = main(text)

서재


NameError: name 'correct_tag' is not defined

In [21]:
# 1번 예시
text = """
자기는 절대 범인이 아니라고 주장한다.
저는 김민수라고 합니다.
이 식물은 선인장이라고 합니다.
이곳은 아름다운 풍경으로 유명한 제주도라고 합니다.
저 책은 올해의 베스트셀러라고 합니다.
"""
final_output = main(text)

형태소 '자기' (위치: 1, 길이: 2, 품사: NP - 대명사)
형태소 '는' (위치: 3, 길이: 1, 품사: JX - 보조사)
형태소 '절대' (위치: 5, 길이: 2, 품사: MAG - 일반 부사)
형태소 '범인' (위치: 8, 길이: 2, 품사: NNG - 일반 명사)
형태소 '이' (위치: 10, 길이: 1, 품사: JKC - 보격 조사)
형태소 '아니' (위치: 12, 길이: 2, 품사: VCN - 부정 지시사(아니다))
형태소 '라고' (위치: 14, 길이: 2, 품사: EC - 연결 어미)
형태소 '주장' (위치: 17, 길이: 2, 품사: NNG - 일반 명사)
형태소 '하' (위치: 19, 길이: 1, 품사: XSV - 동사 파생 접미사)
형태소 'ᆫ다' (위치: 19, 길이: 2, 품사: EF - 종결 어미)
형태소 '.' (위치: 21, 길이: 1, 품사: SF - 종결 부호(. ! ?))
형태소 '저' (위치: 23, 길이: 1, 품사: NP - 대명사)
형태소 '는' (위치: 24, 길이: 1, 품사: JX - 보조사)
형태소 '김민수' (위치: 26, 길이: 3, 품사: NNP - 고유 명사)
형태소 '이' (위치: 29, 길이: 0, 품사: VCP - 긍정 지시사(이다))
형태소 '라고' (위치: 29, 길이: 2, 품사: EC - 연결 어미)
형태소 '하' (위치: 32, 길이: 1, 품사: VV - 동사)
형태소 'ᆸ니다' (위치: 32, 길이: 3, 품사: EF - 종결 어미)
형태소 '.' (위치: 35, 길이: 1, 품사: SF - 종결 부호(. ! ?))
형태소 '이' (위치: 37, 길이: 1, 품사: MM - 관형사)
형태소 '식물' (위치: 39, 길이: 2, 품사: NNG - 일반 명사)
형태소 '은' (위치: 41, 길이: 1, 품사: JX - 보조사)
형태소 '선인장' (위치: 43, 길이: 3, 품사: NNG - 일반 명사)
형태소 '이' (위치: 46, 길이: 1, 품사: VCP - 긍정 지시사(이다))